In [465]:
%matplotlib notebook
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf', 'svg')

import numpy as np
import math as m

from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib.pyplot as plt
from matplotlib import ticker

## Step 1: Function evaluations 
Evaluation of the objective function and its gradient given a specific point $p$, number of segments $n$, segment lenghts $l$ and a set of configuration angles $v$. 

In [466]:
def evalu(n,l,p0,v):
    
    temp0 = np.zeros(n)
    temp1 = np.zeros(n)
    temp2 = np.zeros(n)
    
    f = np.zeros(2)
    g = np.zeros(2)
    ggh = np.zeros((3,2))
    grad_g = np.zeros((3,2))
    
    for i in range(n):
        
        c = m.cos(v[i]+np.sum(temp0))
        s = m.sin(v[i]+np.sum(temp0))
        
        f[0] = (l[i]*c)+f[0]
        f[1] = (l[i]*s)+f[1]
        
        ggh[0,0] = l[i]*(-s)+ggh[0,0]
        ggh[0,1] = l[i]*(c)+ggh[0,1]
        
        temp0[i] = v[i]
        
        
    for i in range(1,n):
        
        temp1[i] = v[i-1]
        
        c = m.cos(v[i]+np.sum(temp1))
        s = m.sin(v[i]+np.sum(temp1))
        
        ggh[1,0] = l[i]*(-s)+ggh[1,0]
        ggh[1,1] = l[i]*(c)+ggh[1,1]
        
        
        
    for i in range(2,n):
        
        temp2[i-1] = v[i-2]
        temp2[i] = v[i-1]
        
        c = m.cos(v[i]+np.sum(temp2))
        s = m.sin(v[i]+np.sum(temp2))
        
        ggh[2,0] = l[i]*(-s)+ggh[2,0]
        ggh[2,1] = l[i]*(c)+ggh[2,1]
        
        
        
    
    g[0] = 0.5*(f[0]-p0[0])**2
    g[1] = 0.5*(f[1]-p0[1])**2
    
    grad_g = ggh @ (f-p)
    
#     for i in range(n):
        
#         for j in range(2):
            
#             if j==0:
# #                 grad_g[i,j] = ggh[i,j]@(f[0]-p0[0])
#                 grad_g = ggh @ ()
#             else:
#                 grad_g[i,j] = ggh[i,j]@(f[1]-p0[1])
            
    
    return (g,grad_g)



        
        
        




## Step 2:Line search
The first thing we will do is to find the line search using "Backtracking Armijo".

In [467]:
def BacktrackLineSearch(vk, gk, ggk, pk, ak, c, rho, nmaxls=100):
    
    #pkggk = np.dot(pk, ggk)
    pkggk = pk @ ggk
    v = vk+(ak*pk)
   
    g,ggk = evalu(n,l,p0,v)
    
    while g[0] <= gk[0] + c*ak*pkggk and g[1] <= gk[1] + c*ak*pkggk:
        ak *= rho
        g,ggk = evalu(n,l,p, v)
        
    return ak

## Step 2:
Then, we need to compute the search direction. For Gradient descent, this is simply $p_{k+1}=-\nabla f(x_{k+1})$, and we have already computed this during the first step.

## Step 3:Optimization
Now we need to compute the search direction, perform line search along this direction and then perform the step.

In [468]:
def optimize(n, l, p0, c, rho, tol):
    
    vk = v0
    v_temp = np.zeros((nmax,3))
    listofAng = np.zeros((nmax,3))
    gk,ggk = evalu(n,l,p0,vk)
    pk = None
    ak = 1
    
    
    for k in range(nmax):
        
        pk = -ggk   #The search direction for steepest descent is simply the gradient
        ak = BacktrackLineSearch(vk, gk, ggk, pk, ak, c, rho, nmaxls=100)
        
    
        v_temp = np.append(vk, vk + np.multiply(ak, pk), axis=0)
        listofAng[k] = vk + np.multiply(ak, pk)
        ggkold=ggk
        gk,ggk = evalu(n,l,p0,listofAng[k]) 
        
        if np.linalg.norm(ggk) < tol:
            break
    
    return listofAng
    

In [ ]:
#Input arguments
n = 3  #Number of segments
l = np.array([3, 2, 2])
p0 = np.array([3, 2])

#Initial conditions
#v0 = np.array([1.72, -1.54, -0.9])
v0 = np.array([0.05, 0.05, 0.05])

#Algorithm parameters
c = 0.5
rho = 0.5
tol = 1e-8
nmax = 100

vgd = optimize(n, l, p0, c, rho, tol)
print(vgd)











In [460]:
###############################################################################################################################

###  Debugging 

############################################################################################################################




# v0 = [m.pi/2, m.pi/2, m.pi/2]
# vk = v0
# gk,ggk = evalu(n,l,p0,vk)
# pk = -ggk
# print("This is ggk:",ggk)
# print(ggk.shape)

# print("This is pk:",pk)
# print(pk.shape)


v_temp = np.zeros((10,3))
lista = np.zeros((10,3))
vk = [1, 1, 1]
ak = 2
pk = [-7, 8, 10] 
v_temp = np.append(vk, vk + np.multiply(ak, pk), axis=0)
lista[0] = vk + np.multiply(ak, pk)

print(lista)
print(v_temp)





# B = np.array([[1, 2], [3, 4], [5, 6]])
# C = np.array([[1, 2], [3, 4], [5, 6]])
# print(B.shape)
# print(C.shape)
# print(B[0])

# D = np.matmul(B[0], C[0])
# print(D)

[[-13.  17.  21.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]] 
[  1   1   1 -13  17  21] 
